In [2]:
import tensorflow as tf

# GPU 사용 여부 체크 및 로그

In [ ]:
# 텐서 연산 시 어떤 장치에서 연산이 일어나고 있는지 로깅
tf.debugging.set_log_device_placement(True)

# 텐서 생성
tensor1 = tf.constant([[1, 1], # constant : 상수. 수정할 수 없음 , variable이 변수를 생성할 때 사용
                       [2, 2]])

tensor2 = tf.constant([[5, 6],
                       [7, 8]])

# 내적
matmul_result = tf.matmul(tensor1, tensor2)
print(matmul_result)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[12 14]
 [24 28]], shape=(2, 2), dtype=int32)


In [ ]:
# 구체적인 장치 정보 확인하기
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15986628300535391101
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2900256699509035550
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

# 텐서 생성하기
- Tensorflow의 텐서(tensor)는 기능적으로 넘파이(numpy)와 매우 유사합니다.
- 기본적으로 **다차원 배열**을 처리하기에 적합한 자료구조로 이해할 수 있습니다.
- Tensorflow의 텐서는 **"자동 미분"**기능을 제공합니다.
  - 역전파, Tensorflow가 계산그래프를 자동으로 기억하고 있음

## 1) 텐서의 속성
- 텐서의 기본 속성은 다음과 같습니다.
  - 모양(shape)
  - 자료형(data type)
  - 저장된 장치

In [ ]:
data = [[1, 2],
        [3, 4]]

tensor = tf.constant(data)

print(tensor)
print(f"Shape : {tensor.shape}")
print(f"Data Type : {tensor.dtype}")
print(f"Device : {tensor.device}")
print(f"Rank : {tf.rank(tensor)}")

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
Shape : (2, 2)
Data Type : <dtype: 'int32'>
Device : /job:localhost/replica:0/task:0/device:GPU:0
Rank : 2


## 2) 텐서 생성하기
- 파이썬의 `list`에서 직접 텐서를 생성할 수 있습니다.

In [ ]:
data = [[1, 2],
        [3, 4]]

x = tf.constant(data)
print(x)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


- numpy 배열에서 텐서를 생성할 수 있습니다.

In [ ]:
a = tf.constant([5])
b = tf.constant([7])

c = (a + b).numpy()
print(type(c))

<class 'numpy.ndarray'>


In [ ]:
result = c * 10
tensor = tf.convert_to_tensor(result)
print(tensor)
print(type(tensor))

tf.Tensor([120], shape=(1,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>


## 3) 다른 텐서로부터 텐서 초기화 하기
- 다른 텐서의 정보를 토대로 텐서를 초기화 할 수 있습니다.
- **텐서의 속성**이란? 모양(shape)과 자료형(data type)을 일컫습니다.

In [ ]:
x = tf.constant([[5, 7],
                 [1, 2]])

# 텐서 x와 같은 모양을 가지지만, 값이 1인 텐서 생성하기
x_ones = tf.ones_like(x)
print(x_ones)

tf.Tensor(
[[1 1]
 [1 1]], shape=(2, 2), dtype=int32)


In [ ]:
# 텐서 x와 같은 모양을 가지지만, 자료형을 float으로, 값은 랜덤하게 채우기
x_rand = tf.random.uniform(x.shape, dtype=tf.float32) # uniform distribution [0, 1]
print(x_rand)

tf.Tensor(
[[0.9896847  0.09121215]
 [0.93899786 0.70856476]], shape=(2, 2), dtype=float32)


# 텐서의 형변환 및 차원 조작
- 텐서는 넘파이 배열처럼 조작이 가능합니다.

## 1) 텐서의 특정 차원에 접근하기
- 텐서의 원하는 차원에 접근할 수 있습니다.

In [ ]:
tensor = tf.constant([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])

print(tensor[0])
print(tensor[:, 0])
print(tensor[..., -1]) # 스프레드 연산 : 특정 차원 사이에 있는 모든 데이터를 가져오고 싶을 때[:,-1]

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor([1 5 9], shape=(3,), dtype=int32)
tf.Tensor([ 4  8 12], shape=(3,), dtype=int32)


## 2) 텐서 이어붙이기
-텐서를 이어서 새로운 텐서를 만들 수 있습니다.

### 2-1) cat
단순하게 텐서를 축(dim)에 맞춰 연결합니다.

In [ ]:
tensor1 = tf.constant([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])

tensor2 = tf.constant([
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 22, 23, 24]
])

# axis : 텐서의 축
# 0번 축을 기준으로 이어 붙입니다.
result = tf.concat([tensor1, tensor2], axis=0)
print(result)

print()
# 1번 축을 기준으로 이어 붙입니다.
result = tf.concat([tensor1, tensor2], axis=1)
print(result)

tf.Tensor(
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]], shape=(6, 4), dtype=int32)

tf.Tensor(
[[ 1  2  3  4 13 14 15 16]
 [ 5  6  7  8 17 18 19 20]
 [ 9 10 11 12 21 22 23 24]], shape=(3, 8), dtype=int32)


### 2-2) stack
리스트 내의 텐서를 쌓아줍니다. 차원수가 증가합니다. 예를 들어 2차원 배열과 2차원 배열을 쌓은 3차원 배열을 만들어 줍니다.

In [ ]:
tensor1 = tf.constant([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])

tensor2 = tf.constant([
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 22, 23, 24]
])

tensor3 = tf.stack([tensor1, tensor2])
tensor3

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

## 3) 텐서 자르기(slice)

In [ ]:
tensor1 = tf.constant([[1,2,3,4],
                        [5,6,7,8]])

tensor2 = tensor1[:, 0]
print(tensor2)

tf.Tensor([1 5], shape=(2,), dtype=int32)


In [ ]:
tensor3 = tensor1[:, 1:3:2]
print(tensor3)

tf.Tensor(
[[2]
 [6]], shape=(2, 1), dtype=int32)


In [ ]:
tensor3 = tensor1[0, 1:3]
print(tensor3)

tf.Tensor([2 3], shape=(2,), dtype=int32)


In [ ]:
tensor1 = tf.constant([[[1, 2],
                         [3, 4]],
                  
                        [[5, 6],
                         [7, 8]],
                  
                        [[9, 10],
                         [11,12]]])

print(tensor1[0])
print(tensor1[0, :])
print(tensor1[0, :, 0])

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor([1 3], shape=(2,), dtype=int32)


In [ ]:
print(tensor1[-1])
print(tensor1[-1, :])
print(tensor1[-1, :, :])

tf.Tensor(
[[ 9 10]
 [11 12]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 9 10]
 [11 12]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 9 10]
 [11 12]], shape=(2, 2), dtype=int32)


In [ ]:
print(tensor1[1:3, :, :].shape)
print(tensor1[:, 1, :].shape)
print(tensor1[:, :-1, :].shape)

(2, 2, 2)
(3, 2)
(3, 1, 2)


### 3-1) split
지정한 size만큼 tensor를 쪼개어 줍니다.

In [ ]:
tensor1 = tf.random.uniform((16, 4))

splits = tf.split(tensor1, num_or_size_splits=4, axis=0)
for s in splits:
  print(s.shape)

(4, 4)
(4, 4)
(4, 4)
(4, 4)


### 3-2) gather
인덱스를 이용해 해당 위치의 텐서를 참조합니다.

In [ ]:
tensor1 = tf.constant([[[1, 2],
                         [3, 4]],
                  
                        [[5, 6],
                         [7, 8]],
                  
                        [[9, 10],
                         [11,12]]])

indice = tf.constant([2, 0])
print(indice)

tensor2 = tf.gather(tensor1, indice, axis=0)
print(tensor2)

tf.Tensor([2 0], shape=(2,), dtype=int32)
tf.Tensor(
[[[ 9 10]
  [11 12]]

 [[ 1  2]
  [ 3  4]]], shape=(2, 2, 2), dtype=int32)


## 4) 텐서 형변환(type casting)
- 텐서의 자료형을 변환할 수 있습니다.

In [ ]:
tensor1 = tf.constant([2])
tensor2 = tf.constant([5.0])

print(tensor1.dtype)
print(tensor2.dtype)

<dtype: 'int32'>
<dtype: 'float32'>


In [ ]:
print(tf.cast(tensor1, tf.float32) + tensor2)
print((tf.cast(tensor1, tf.float32) + tensor2).dtype)

tf.Tensor([7.], shape=(1,), dtype=float32)
<dtype: 'float32'>


In [ ]:
print((tensor1 + tf.cast(tensor2, tf.int32)))

tf.Tensor([7], shape=(1,), dtype=int32)


## 5) 텐서의 모양 변경

### 5-1) reshape
- numpy의 reshape과 동일하며, 텐서의 순서가 변경되지 않습니다.

In [ ]:
tensor1 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8])

tensor2 = tf.reshape(tensor1, (4, 2))
print(tensor2)

tf.Tensor(
[[1 2]
 [3 4]
 [5 6]
 [7 8]], shape=(4, 2), dtype=int32)


In [ ]:
# 이 방식은 불가
tensor2 = tensor1.reshape(4, 2)

print(f"tensor2 : {tensor2}")

AttributeError: ignored

### 5-2) transpose
하나의 텐서에서 특정한 차원끼리 순서를 교체할 수 있습니다.

In [ ]:
tensor1 = tf.random.uniform((3, 2, 5))
print(tensor1.shape)

tensor2 = tf.transpose(tensor1, perm=[2, 1, 0]) # perm : 바꿀 axis 목록
print(tensor2.shape)

# 텐서의 연산

In [3]:
a = tf.constant([[1, 2],
                  [3, 4]])

b = tf.constant([[2, 2],
                  [3, 3]])

In [4]:
print(a + b)

tf.Tensor(
[[3 4]
 [6 7]], shape=(2, 2), dtype=int32)


In [ ]:
print(a - b)

In [ ]:
print(a * b)

In [ ]:
print(a / b)

In [ ]:
print( a == b )

In [ ]:
print( a != b )

In [ ]:
print( a ** b )

## Inplace 연산
Inplace 연산은 텐서의 실제 값을 바꿔주는 연산을 의미합니다. 이 때 텐서의 값 자체가 바뀌어야 한다면 tf.Variable을 이용할 수 있습니다.

In [ ]:
a = tf.Variable([[1, 2],
                  [3, 4]])

b = tf.constant([[2, 2],
                  [3, 3]])

print(a)
print(a.assign_add(b)) # 텐서 a의 값 자체가 바뀌는 inplace 연산
print(a)

In [ ]:
a = tf.constant([[1, 2],
                  [3, 4]])

b = tf.constant([[2, 2],
                  [3, 3]])

print(a + b)
print(a)

## 차원 감소 연산
주로 합계나 평균 등 차원 축에 따라 연산이 된 후 감소가 되는 연산을 의미합니다.
- 집계 : Aggregation => Reduction

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]], dtype=tf.float32)

# 차원을 따로 지정하지 않으면 텐서 내 모든 데이터에 대한 연산이 이뤄진 후 스칼라 값으로 표현
print(tf.math.reduce_sum(a))
print(tf.math.reduce_mean(a))

In [ ]:
# 차원을 따로 지정하면, 지정한 차원에 맞게 연산
print(tf.math.reduce_sum(a,axis=0))
print(tf.math.reduce_sum(a,axis=1))

print()

print(tf.math.reduce_mean(a,axis=0))
print(tf.math.reduce_mean(a,axis=1))

## 브로드캐스팅
차원 수가 맞지 않아도 자동으로 텐서를 높은 차원으로 확장해 연산이 가능토록 합니다.

In [ ]:
a = tf.constant([[1, 2]])
b = tf.constant([[3, 4]])

print(a.shape, b.shape)

In [ ]:
print(a + b)

### tensor, scalar 연산

In [ ]:
a = tf.constant([[1, 2],
                  [3, 4]])

print(a + 10)

### tensor, vector 연산

In [5]:
a = tf.constant([[1, 2],
                  [3, 4]])

b = tf.constant([10, 20])

print(b.shape)
print(a + b)

(2,)
tf.Tensor(
[[11 22]
 [13 24]], shape=(2, 2), dtype=int32)


In [ ]:
a = tf.constant([[1, 2],
                  [3, 4]])

b = tf.constant([[10, 20]])
print(b.shape)
print(a + b)

In [ ]:
a = tf.constant([[1, 2],
                  [3, 4]])

b = tf.constant([[10],
                  [20]])
print(b.shape)
print(a + b)

### tensor, tensor 연산

In [6]:
a = tf.constant([[1, 2]])

b = tf.constant([[10],
                  [20]])

print(a.shape, b.shape)
print(a + b)

(1, 2) (2, 1)
tf.Tensor(
[[11 12]
 [21 22]], shape=(2, 2), dtype=int32)


In [7]:
a = tf.constant([[[1, 2],
                   [3, 4]],
                  
                   [[5, 6],
                    [7, 8]],
                  
                   [[9, 10],
                    [11,12]]])

b = tf.constant([100, 200])

print(a+b)

tf.Tensor(
[[[101 202]
  [103 204]]

 [[105 206]
  [107 208]]

 [[109 210]
  [111 212]]], shape=(3, 2, 2), dtype=int32)


In [ ]:
a = tf.constant([[[1, 2],
                   [3, 4]],
                  
                   [[5, 6],
                    [7, 8]],
                  
                   [[9, 10],
                    [11,12]]])

b = tf.constant([[100, 200],
                  [300, 400]])

print(a+b)

In [ ]:
a = tf.constant([[[1, 2],
                   [3, 4]],
                  
                   [[5, 6],
                    [7, 8]],
                  
                   [[9, 10],
                    [11,12]]])

b = tf.constant([[100],
                  [300]])

print(a+b)

# 텐서플로우의 여러 함수들

## expand_dims
설정한 축의 차원 수를 1 늘려줍니다.

In [ ]:
x = tf.constant([[[1, 2]],
                  [[3, 4]]])

print(x.shape) # x.shape과 같다.

(2, 1, 2)


In [ ]:
y = tf.expand_dims(x, 0)
print(y)
print(y.shape)

tf.Tensor(
[[[[1 2]]

  [[3 4]]]], shape=(1, 2, 1, 2), dtype=int32)
(1, 2, 1, 2)


## argmax
설정한 차원에서 가장 큰 값이 들어있는 인덱스를 반환합니다.

In [ ]:
x = tf.constant([[1,5,9,2],
                 [3,4,6,7]])

print(x)
print(x.shape)

tf.Tensor(
[[1 5 9 2]
 [3 4 6 7]], shape=(2, 4), dtype=int32)
(2, 4)


In [ ]:
# -1번 방향(스칼라 축)에서 가장 큰 값의 인덱스를 추출하기
y = tf.argmax(x, axis=-1) # 마지막 차원이므로 행벡터

print(y)
print(y.shape)

tf.Tensor([2 3], shape=(2,), dtype=int64)
(2,)


In [ ]:
# 0번 방향(1차원 벡터 축)에서 가장 큰 값의 인덱스를 추출하기
y = tf.argmax(x, axis=0)

print(y)
print(y.shape)

tf.Tensor([1 0 0 1], shape=(4,), dtype=int64)
(4,)


## top_k
가장 큰 k개의 값과, 그 값의 인덱스를 튜플 형식으로 반환합니다.

In [ ]:
values, indices = tf.math.top_k(x, k=1)

print(values)
print(values.shape)

tf.Tensor(
[[9]
 [7]], shape=(2, 1), dtype=int32)
(2, 1)


In [ ]:
print(indices)
print(indices.shape)

tf.Tensor(
[[2]
 [3]], shape=(2, 1), dtype=int32)
(2, 1)


### topk를 이용해 정렬하기

In [ ]:
target_dim = -1
values, indices = tf.math.top_k(
    x,
    k=x.shape[target_dim],
)

print(values)

tf.Tensor(
[[9 5 2 1]
 [7 6 4 3]], shape=(2, 4), dtype=int32)


## squeeze

- squeeze : 개수가 하나인 차원을 삭제합니다.

In [ ]:
# squeeze
x = tf.constant([[[1, 2],
                   [3, 4]]])

print(x)
print(x.shape)

tf.Tensor(
[[[1 2]
  [3 4]]], shape=(1, 2, 2), dtype=int32)
(1, 2, 2)


In [ ]:
print(tf.squeeze(x))
print(tf.squeeze(x).shape)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
(2, 2)


In [ ]:
print(tf.squeeze(x, axis=0))
print(tf.squeeze(x, axis=0).shape)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
(2, 2)


In [ ]:
# # 차원수가 1이 아닌 위치의 axis에서는 오류가 발생된다.
# print(tf.squeeze(x, axis=1))
# print(tf.squeeze(x, axis=1).shape) # 인덱스 1번 위치에는 element가 2개 이므로 squeeze가 일어나지 않는다.